In [ ]:

 from sense_hat import SenseHat
sense = SenseHat()

t = sense.get_temperature()
h = sense.get_humidity()
p = sense.get_pressure()
a = sense.get_accelerometer_raw()
g = sense.get_gyroscope_raw()
m = sense.get_compass_raw()

print(f"Temp: {t:.1f} °C")
print(f"Hum : {h:.1f} %")
print(f"Pres: {p:.1f} hPa")

print(f"Accel: {a['x']:.2f}g, {a['y']:.2f}g, {a['z']:.2f}g")
print(f"Gyro : {g['x']:.2f}°/s, {g['y']:.2f}°/s, {g['z']:.2f}°/s")
print(f"Mag  : {m['x']:.2f}µT, {m['y']:.2f}µT, {m['z']:.2f}µT")


In [ ]:
from sense_hat import SenseHat
import matplotlib.pyplot as plt
import time

sense = SenseHat()

accel_x = []
gyro_x = []
mag_x = []
time_data = []

plt.ion()
fig, ax_plot = plt.subplots()

start = time.time()

while True:
    accel = sense.get_accelerometer_raw()
    gyro = sense.get_gyroscope_raw()
    mag = sense.get_compass_raw()

    now = time.time() - start
    time_data.append(now)
    accel_x.append(accel['x'])
    gyro_x.append(gyro['x'])
    mag_x.append(mag['x'])

    ax_plot.clear()
    ax_plot.plot(time_data, accel_x, label="Accel X")
    ax_plot.plot(time_data, gyro_x, label="Gyro X")
    ax_plot.plot(time_data, mag_x, label="Mag X")

    ax_plot.legend()
    ax_plot.set_xlabel("Time (s)")
    ax_plot.set_ylabel("Value")

    plt.pause(0.1)  # Fixed: removed space, 10 updates per second

# Analysis of the assignment

We want to predict the position of a pedestrian using a floorplan and refraining from using GPS. We can use a IMU which produces acceleration, and orientation data. (It also produces temperature, compass 
, pressure and humidity data but those are probably not relevant to the problem we are trying to solve.)

We should use a non-trivial floorplan in buildings or carparkingdeck while we avoid entering the actual parking spaces. We will need to measure this floorplan or obtain the drawings of the building.

The paper assumes there is a detector that tells you that a stride took place and at the same time gives you the change in orientation of the IMU (in this case the orientation of the RPi as the IMU is an integral part of it.) This sensor information is used in a Bayesian inference
 step to estimate the location.
 
 So in high level pseudo code, we want to achieve the following:
 
 ```{r, eval=FALSE,tidy=FALSE}
 repeat:
     timestamp = wait_for_stride_event  # initially button press, later infered from accelerometer
     orientation = read gyroscope
     loc = infer_most_likely_location(loc, orientation)  # using a variant of eq. 5
     publish_or_store_location(loc, timestamp)
```     

From this code we can see that we need to specify an initial condition, because `infer_most_likely_location` takes the previous location as an input argument. Also it is at present not entirely clear whether location is a single point or a probability distribution. Koroglu and Yilmaz write about using a mode seeking algorithm, the paper is a bit vague about whether this is used only for producing a single definite answer at each time step or also used as only input to the next step.

Other problem we can see with this pseudocode is that the orientation coming from the SenseHat is relative to its initial orientation, not relative to the map. So again we need to initialize make an assumption about the initial orienation. 

The assumptions we make here influence how we can use our code later. If we assume an initial location and orientation we need to boot the RPi at this location and orientation during testing! We wil make this simplifying assumption anyway, and our pseudocode becomes:

```{r, eval=FALSE,tidy=FALSE}
 '''
 When using this code you need to start at the initial location and the length RPi should be aligned with the arrow on the floorplan. When moving in direction of the arrow on the floorplan it corresponds with varying the second index in our array representing it.
 '''

 loc = initial_location  
 
 repeat:
     timestamp = wait_for_stride_event()  # initially button press, later infered from accelerometer
     orientation = read gyroscope()
     loc = infer_most_likely_location(loc, orientation)  # using a variant of eq. 5
     publish_or_store_location(loc, timestamp)
```     

This is the point to start gathering bit and pieces of working code.
 

In [1]:
''' wait for stride event: button press on SenseHat '''
from sense_hat import SenseHat
import time

sense = SenseHat()

button_pressed = False

while not button_pressed:
    for event in sense.stick.get_events():
        print(event.direction, event.action)
        if event.direction == 'middle' and event.action == 'released':
            button_pressed = True
            
# This works lets wrap it in a function

def wait_for_stride_event(usemousebutton=False):
    if usemousebutton:
        button_pressed = False

        while not button_pressed:
            for event in sense.stick.get_events():
                print(event.direction, event.action)  # Remove later but it provides usefull feedback now
                if event.direction == 'middle' and event.action == 'released':  # Add action released otherwise release event is handled in next function call
                    button_pressed = True
    else:
        raise NotImplementedError('This function (wait_for_stride_event) is still a stub, if you set usemousebutton=True it can be used for testing')

    return time.time()
           
# Check the mouse version
wait_for_stride_event(usemousebutton=True)   
        
# Check our Exception        
try:      
    wait_for_stride_event()   
except NotImplementedError as e:
    print(e)

 

middle pressed
middle released
middle pressed
middle released
This function (wait_for_stride_event) is still a stub, if you set usemousebutton=True it can be used for testing


In [ ]:
sense.set_pixel(0, 0, (0,0,0))
sense.set_pixel(0, 7, (0,0,0))

In [ ]:
''' Reading the IMU data'''

from sense_hat import SenseHat
import time
import numpy as np

sense = SenseHat()

sense.set_imu_config(True, True, False)  # acceleration: On, Gyroscope: On, Compass: Off
t_start = time.time()

pixel_sets = {0: [[0,3],[0,4]],
              1: [[0,1],[0,0],[1,0]],
              2: [[3,0],[4,0]],
              3: [[6,0],[7,0],[7,1]],
              4: [[7,3],[7,4]],
              5: [[7,6],[7,7],[6,7]],
              6: [[3,7],[4,7]],
              7: [[1,7],[0,7],[0,6]],}

previous_pixel_set_key = 0

def update_leds(yaw, previous_pixel_set_key,shift=0):
    pixel_set_key = (int(yaw/(.25*np.pi))+shift)%8
    
    for pixel in pixel_sets[previous_pixel_set_key ]:
        sense.set_pixel(pixel[0], pixel[1], (0,0,0))
    
    for pixel in pixel_sets[pixel_set_key]:
        sense.set_pixel(pixel[0], pixel[1], (50,50,0))
        
    return pixel_set_key
    
while time.time() < t_start+120:
    acceleration = sense.get_accelerometer_raw()
    orientation = sense.get_gyroscope()  # get_gyroscope probably bypasses filtering in RTIMU library
                                         # get_orientation gives poor results
    #x = acceleration['x']
    #y = acceleration['y']
    #z = acceleration['z']

    pitch = orientation["pitch"]/360*2*np.pi
    roll = orientation["roll"]/360*2*np.pi
    yaw = orientation["yaw"]/360*2*np.pi

    previous_pixel_set_key = update_leds(-yaw, previous_pixel_set_key,shift=3)

    # print(f"pitch: {pitch}, roll: {roll}, yaw: {yaw}, pixel_set_key: {pixel_set_key}")

sense.clear([0,0,0])  
print('Done!')

In [ ]:
'''Test pixel activatio '''
previous_pixel_set_key=0
for yaw in (np.pi*np.arange(0,32)+.0125*np.pi)/4:
    previous_pixel_set_key = update_leds(yaw, previous_pixel_set_key)
    time.sleep(.5)
sense.clear([0,0,0])